In [30]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
train_df = pd.read_csv('./data/stsb-en-train.csv', usecols=[0,1,2], names=['sent1', 'sent2', 'score'], header=None)

In [12]:
train_df.head()

,sent1,sent2,score
0,A plane is taking off.,An air plane is taking off.,5.00
1,A man is playing a large flute.,A man is playing a flute.,3.80
2,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...,3.80
3,Three men are playing chess.,Two men are playing chess.,2.60
4,A man is playing the cello.,A man seated is playing the cello.,4.25


In [13]:
import re
pattern = r'[^\w\s]'

In [14]:
train_df['sent1'] =train_df['sent1'].apply(lambda x: re.sub(pattern, '', x))
train_df['sent2'] =train_df['sent2'].apply(lambda x: re.sub(pattern, '', x))

In [15]:
train_df.head()

,sent1,sent2,score
0,A plane is taking off,An air plane is taking off,5.00
1,A man is playing a large flute,A man is playing a flute,3.80
2,A man is spreading shreded cheese on a pizza,A man is spreading shredded cheese on an uncoo...,3.80
3,Three men are playing chess,Two men are playing chess,2.60
4,A man is playing the cello,A man seated is playing the cello,4.25


In [16]:
val_df = pd.read_csv('./data/stsb-en-dev.csv', usecols=[0,1,2], names=['sent1', 'sent2', 'score'], header=None)

In [17]:
val_df.head()

,sent1,sent2,score
0,A man with a hard hat is dancing.,A man wearing a hard hat is dancing.,5.00
1,A young child is riding a horse.,A child is riding a horse.,4.75
2,A man is feeding a mouse to a snake.,The man is feeding a mouse to the snake.,5.00
3,A woman is playing the guitar.,A man is playing guitar.,2.40
4,A woman is playing the flute.,A man is playing a flute.,2.75


In [18]:
val_df['sent1'] =val_df['sent1'].apply(lambda x: re.sub(pattern, '', x))
val_df['sent2'] =val_df['sent2'].apply(lambda x: re.sub(pattern, '', x))

In [19]:
val_df.head()

,sent1,sent2,score
0,A man with a hard hat is dancing,A man wearing a hard hat is dancing,5.00
1,A young child is riding a horse,A child is riding a horse,4.75
2,A man is feeding a mouse to a snake,The man is feeding a mouse to the snake,5.00
3,A woman is playing the guitar,A man is playing guitar,2.40
4,A woman is playing the flute,A man is playing a flute,2.75


In [70]:
test_df = pd.read_csv('./data/stsb-en-test.csv', usecols=[0,1,2], names=['sent1', 'sent2', 'score'], header=None)

In [71]:
test_df.head()

,sent1,sent2,score
0,A girl is styling her hair.,A girl is brushing her hair.,2.5
1,A group of men play soccer on the beach.,A group of boys are playing soccer on the beach.,3.6
2,One woman is measuring another woman's ankle.,A woman measures another woman's ankle.,5.0
3,A man is cutting up a cucumber.,A man is slicing a cucumber.,4.2
4,A man is playing a harp.,A man is playing a keyboard.,1.5


In [72]:
test_df['sent1'] =test_df['sent1'].apply(lambda x: re.sub(pattern, '', x))
test_df['sent2'] =test_df['sent2'].apply(lambda x: re.sub(pattern, '', x))

In [20]:
len(train_df)

5749

In [21]:
len(val_df)

1500

In [73]:
len(test_df)

1379

In [22]:
total_sents = list(train_df['sent1'])

In [23]:
total_sents.extend(list(train_df['sent2']))

In [26]:
total_sents[0:5]

['A plane is taking off',
 'A man is playing a large flute',
 'A man is spreading shreded cheese on a pizza',
 'Three men are playing chess',
 'A man is playing the cello']

In [27]:
len(total_sents)

11498

In [28]:
total_sents_df = pd.Series(total_sents)

In [29]:
len(total_sents_df)

11498

In [466]:
vectorizer = TfidfVectorizer()

In [467]:
X_train_sents = vectorizer.fit_transform(total_sents_df)

In [468]:
X_train_sentence1 = vectorizer.transform(train_df['sent1'])
X_train_sentence2 = vectorizer.transform(train_df['sent2'])

In [469]:
X_test_sentence1 = vectorizer.transform(val_df['sent1'])
X_test_sentence2 = vectorizer.transform(val_df['sent2'])

In [470]:
# Combine the sentence vectors for training and testing data
X_train_vectorized = np.hstack((X_train_sentence1.toarray(), X_train_sentence2.toarray()))
X_test_vectorized = np.hstack((X_test_sentence1.toarray(), X_test_sentence2.toarray()))

In [471]:
y_train = train_df['score']
y_test = val_df['score']

In [472]:
model = LinearRegression()
model.fit(X_train_vectorized, y_train)

LinearRegression()

In [473]:
score = model.score(X_test_vectorized, y_test)
print(f"Model score on testing dataset: {score}")

Model score on testing dataset: -3.5885542881675076e+24


In [474]:
model.predict(X_test_vectorized)

array([ 1.12254858e+12,  1.79777806e+12,  1.35155836e+11, ...,
        3.58372417e+12, -7.77053917e+11,  9.39278380e+11])

In [475]:
# Normalize the target variable to be in the range of 0 to 1
y_train_normalized = (y_train - y_train.min()) / (y_train.max() - y_train.min())

# Train the model on the normalized target variable
model = LinearRegression()
model.fit(X_train_vectorized, y_train_normalized)

LinearRegression()

In [476]:
# Predict normalized similarity scores for the testing dataset
y_pred_normalized = model.predict(X_test_vectorized)

# Scale the normalized predictions back to the original range of 0 to 5
y_pred = y_pred_normalized * (y_train.max() - y_train.min()) + y_train.min()

# Evaluate the model's performance on the testing dataset
score = model.score(X_test_vectorized, y_test)
print(f"Model score on testing dataset: {score}")

Model score on testing dataset: -1.4354217152668285e+23


In [477]:
from scipy.stats import pearsonr
def pearson_corr(y_true, y_pred):
    """
    Calculate Pearson correlation coefficient between two arrays.
    """
    corr, _ = pearsonr(y_true, y_pred)
    return corr

In [478]:
# Calculate Pearson correlation coefficient between predicted values and target values
corr = pearson_corr(y_test, y_pred)

# Print the correlation coefficient
print("Pearson correlation coefficient: {:.2f}".format(corr))

Pearson correlation coefficient: 0.01


## Word2vec - Doc2vec

In [13]:
total_sents[0].split()

['A', 'plane', 'is', 'taking', 'off.']

In [40]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from scipy import spatial

In [252]:
max_length = max(len(sentence) for sentence in total_sents)

In [253]:
max_length

360

In [32]:
documents = []
for idx, sent in enumerate(total_sents):
    tokens = sent.split()
    documents.append(TaggedDocument(tokens, [idx]))
    
    

In [34]:
documents[0:5]

[TaggedDocument(words=['A', 'plane', 'is', 'taking', 'off'], tags=[0]),
 TaggedDocument(words=['A', 'man', 'is', 'playing', 'a', 'large', 'flute'], tags=[1]),
 TaggedDocument(words=['A', 'man', 'is', 'spreading', 'shreded', 'cheese', 'on', 'a', 'pizza'], tags=[2]),
 TaggedDocument(words=['Three', 'men', 'are', 'playing', 'chess'], tags=[3]),
 TaggedDocument(words=['A', 'man', 'is', 'playing', 'the', 'cello'], tags=[4])]

In [447]:
# Build and train the Doc2Vec model - This needs further fine tuning
model = Doc2Vec(documents, vector_size=50, window=4, min_count=1, workers=4, epochs=10, alpha=0.025, min_alpha=0.025)

In [314]:
# Save the trained model to disk
#model.save('doc2vec_model_v1')

# Load the saved model from disk
#model = Doc2Vec.load('doc2vec_model_v1')

In [418]:
model.random.seed(42)

In [464]:
# Generate sentence embeddings
embedding1 = model.infer_vector(train_df['sent1'][0].split())
embedding2 = model.infer_vector(train_df['sent2'][0].split())

# Compare the sentence embeddings using cosine similarity
similarity = 1 - spatial.distance.cosine(embedding1, embedding2)

In [465]:
similarity

0.7302546501159668

In [421]:
len(embedding2)

50

In [422]:
len(embedding1)

50

In [423]:
def sts_score(sim_score):
    sts_score = (sim_score+1) * 2.5
    return sts_score

In [424]:
print(sts_score(similarity))

4.768485873937607


In [425]:
model.random.seed(42)
train_df['sent1_embedding'] = train_df['sent1'].apply(lambda x: model.infer_vector(x.split()))
train_df['sent2_embedding'] = train_df['sent2'].apply(lambda x: model.infer_vector(x.split()))

In [426]:
train_df['y_pred'] = train_df.apply(lambda x: sts_score(1 - spatial.distance.cosine(x['sent1_embedding'], x['sent2_embedding'])), axis=1)

In [427]:
train_df.head()

,sent1,sent2,score,sent1_embedding,sent2_embedding,y_pred
0,A plane is taking off,An air plane is taking off,5.00,"[-0.03261799, -0.057231516, -0.025054844, 0.02...","[-0.043522693, -0.08055471, 0.0068473397, 0.06...",4.652529
1,A man is playing a large flute,A man is playing a flute,3.80,"[0.16988525, -0.042989817, 0.040764358, -0.065...","[0.069118336, -0.001824693, 0.03987648, -0.073...",4.703555
2,A man is spreading shreded cheese on a pizza,A man is spreading shredded cheese on an uncoo...,3.80,"[-0.009786395, -0.058945123, 0.08580946, -0.15...","[0.010572702, -0.076822095, 0.07916778, -0.051...",4.748430
3,Three men are playing chess,Two men are playing chess,2.60,"[-0.11417733, 0.05027048, 0.0027939323, 0.0411...","[-0.044409215, -0.0064910166, 0.020731455, 0.0...",4.451858
4,A man is playing the cello,A man seated is playing the cello,4.25,"[0.062338647, 0.082752384, -0.044882916, -0.07...","[-0.08560305, 0.1209445, 0.01598528, -0.050783...",3.531286


In [428]:
y_pred = train_df['y_pred']

In [429]:
y_train = train_df['score']

In [430]:
# Calculate Pearson correlation coefficient between predicted values and target values
corr = pearson_corr(y_train, y_pred)

# Print the correlation coefficient
print("Pearson correlation coefficient: {:.2f}".format(corr))

Pearson correlation coefficient: 0.24


In [431]:
model.random.seed(42)
val_df['sent1_embedding'] = val_df['sent1'].apply(lambda x: model.infer_vector(x.split()))
val_df['sent2_embedding'] = val_df['sent2'].apply(lambda x: model.infer_vector(x.split()))

In [432]:
val_df['y_pred'] = val_df.apply(lambda x: sts_score(1 - spatial.distance.cosine(x['sent1_embedding'], x['sent2_embedding'])), axis=1)

In [433]:
val_df.head()

,sent1,sent2,score,sent1_embedding,sent2_embedding,y_pred
0,A man with a hard hat is dancing,A man wearing a hard hat is dancing,5.00,"[0.102311775, 0.039187357, 0.082304984, -0.108...","[0.11590858, -0.06448609, -0.0059869844, -0.15...",4.551783
1,A young child is riding a horse,A child is riding a horse,4.75,"[0.11334487, -0.035400495, 0.017929595, -0.075...","[0.13285376, -0.09502117, -0.009272606, -0.024...",4.787501
2,A man is feeding a mouse to a snake,The man is feeding a mouse to the snake,5.00,"[0.05396892, -0.060374834, 0.08223716, -0.0879...","[-0.012262808, -0.11392646, 0.055586256, 0.005...",4.094796
3,A woman is playing the guitar,A man is playing guitar,2.40,"[0.106308825, 0.036144402, -0.026240667, -0.05...","[-0.0056881197, -0.004985351, 0.063051865, 0.0...",3.172627
4,A woman is playing the flute,A man is playing a flute,2.75,"[0.08801661, 0.0036291284, 0.031732973, -0.100...","[0.048949435, 0.0021717984, 0.025101138, -0.04...",4.743179


In [434]:
y_pred = val_df['y_pred']
y_val = val_df['score']

In [435]:
# Calculate Pearson correlation coefficient between predicted values and target values
corr = pearson_corr(y_val, y_pred)

# Print the correlation coefficient
print("Pearson correlation coefficient: {:.2f}".format(corr))

Pearson correlation coefficient: 0.32


In [436]:
model.random.seed(42)
test_df['sent1_embedding'] = test_df['sent1'].apply(lambda x: model.infer_vector(x.split()))
test_df['sent2_embedding'] = test_df['sent2'].apply(lambda x: model.infer_vector(x.split()))

In [437]:
test_df['y_pred'] = test_df.apply(lambda x: sts_score(1 - spatial.distance.cosine(x['sent1_embedding'], x['sent2_embedding'])), axis=1)

In [438]:
test_df.head()

,sent1,sent2,score,sent1_embedding,sent2_embedding,y_pred
0,A girl is styling her hair,A girl is brushing her hair,2.5,"[0.047453664, -0.012963152, 0.01766814, -0.132...","[0.05282152, -0.032737095, 0.027383164, -0.079...",4.314512
1,A group of men play soccer on the beach,A group of boys are playing soccer on the beach,3.6,"[-0.005135766, 0.11651001, -0.058646847, -0.09...","[0.051796418, 0.020778747, -0.05155143, -0.139...",4.624106
2,One woman is measuring another womans ankle,A woman measures another womans ankle,5.0,"[0.041873578, -0.041858833, -0.0081949495, -0....","[0.11333422, 0.002360624, -0.064282775, -0.126...",4.349280
3,A man is cutting up a cucumber,A man is slicing a cucumber,4.2,"[0.11555635, -0.07165362, 0.08308311, -0.00472...","[0.097490035, -0.014397144, 0.04787491, -0.088...",4.289850
4,A man is playing a harp,A man is playing a keyboard,1.5,"[0.0689409, 0.0052396706, 0.054995388, -0.0906...","[-0.00028628367, 0.03726319, 0.032384034, -0.0...",4.586179


In [439]:
y_pred = test_df['y_pred']
y_test = test_df['score']

In [440]:
# Calculate Pearson correlation coefficient between predicted values and target values
corr = pearson_corr(y_test, y_pred)

# Print the correlation coefficient
print("Pearson correlation coefficient: {:.2f}".format(corr))

Pearson correlation coefficient: 0.18
